# SDPD Crime Data Latitude and Longitude Extraction

## Table of Contents:
* [Libraries](#libraries)
* [Crime Data Zips Object](#crime_data_object)

### Libraries  <a class="anchor" id="libraries"></a>

In [1]:
import pandas as pd
import requests
import numpy as np

### Crime Data Zips Object  <a class="anchor" id="crime_data_object"></a>

In [2]:
class Crime_Data_Zips: 
    def __init__(self):
        self.google_maps_key='Please Add in Your Custom Key'
    
    @staticmethod
    def read_in_crime_data():
        stops=pd.read_csv("gs://dse203_group_project/Data/SDPD/police_data/ripa_stops_datasd_v1.csv")
        race=pd.read_csv("gs://dse203_group_project/Data/SDPD/police_data/ripa_race_datasd.csv")
        reason_for_stop=pd.read_csv("gs://dse203_group_project/Data/SDPD/police_data/ripa_stop_reason_datasd.csv")
        result=pd.read_csv("gs://dse203_group_project/Data/SDPD/police_data/ripa_stop_result_datasd.csv")
        complaints=pd.read_csv("gs://dse203_group_project/Data/SDPD/police_data/complaints_sdpd.csv")
        
        stops_race=pd.merge(race, stops)
        stops_comp=pd.merge(stops_race, reason_for_stop)
        stops_comp=pd.merge(stops_comp, result)
        
        test=stops_comp[stops_comp['stop_in_response_to_cfs']==1]
        test=test[test['reason_for_stop']!='Traffic Violation']
        test=test[test['reason_for_stopcode']!='54106']
        test=test[test['reason_for_stopcode']!=54106]
        test=test[test['reason_for_stop_explanation']!='dog unrestrained']

        test=test.dropna(subset=['address_street', 'address_city'])
        test.reset_index(inplace=True, drop=True)
        
        return test
        
    def lon_lat(self):
        test=self.read_in_crime_data()
        
        zips_df=test[['address_street', 'address_city']]
        zips_df['state']='CA'
        zips_df['full_address']=zips_df['address_street']+' '+zips_df['address_city']+' '+zips_df['state']
        zips_df['full_address']=zips_df['full_address'].str.strip()
        zips_df['full_address']=zips_df['full_address'].replace(' ', '+', regex=True)
        
        full_address_list=zips_df['full_address'].values.tolist()
        
        
        # Extacting lat_long using Google API
        latitude=[]
        longitude=[]
        i=1
        for address in full_address_list:
            response = requests.get(f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={self.google_maps_key}')
            resp_json_payload = response.json()

            try:
                latitude.append(resp_json_payload['results'][0]['geometry']['location']['lat'])
                longitude.append(resp_json_payload['results'][0]['geometry']['location']['lng'])
            except:
                latitude.append(np.nan)
                longitude.append(np.nan)

            print(f'{i}/{len(full_address_list)}')
            i=i+1
        
        zips_df['longitude']=longitude
        zips_df['latitude']=latitude
        
        test=test.merge(zips_df[['longitude','latitude']], how='left', left_index=True, right_index=True)
        
        return test
               

In [3]:
crime_data_zips=Crime_Data_Zips()
df=crime_data_zips.lon_lat()
df.to_csv('gs://dse203_group_project/Data/sdpd_geo_location.csv', index=False)

/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3433: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3433: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_12528/2246424491.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zips_df['state']='CA'
/tmp/ipykernel_12528/2246424491.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

1/26633
2/26633
3/26633
4/26633
5/26633
6/26633
7/26633
8/26633
9/26633
10/26633
11/26633
12/26633
13/26633
14/26633
15/26633
16/26633
17/26633
18/26633
19/26633
20/26633
21/26633
22/26633
23/26633
24/26633
25/26633
26/26633
27/26633
28/26633
29/26633
30/26633
31/26633
32/26633
33/26633
34/26633
35/26633
36/26633
37/26633
38/26633
39/26633
40/26633
41/26633
42/26633
43/26633
44/26633
45/26633
46/26633
47/26633
48/26633
49/26633
50/26633
51/26633
52/26633
53/26633
54/26633
55/26633
56/26633
57/26633
58/26633
59/26633
60/26633
61/26633
62/26633
63/26633
64/26633
65/26633
66/26633
67/26633
68/26633
69/26633
70/26633
71/26633
72/26633
73/26633
74/26633
75/26633
76/26633
77/26633
78/26633
79/26633
80/26633
81/26633
82/26633
83/26633
84/26633
85/26633
86/26633
87/26633
88/26633
89/26633
90/26633
91/26633
92/26633
93/26633
94/26633
95/26633
96/26633
97/26633
98/26633
99/26633
100/26633
101/26633
102/26633
103/26633
104/26633
105/26633
106/26633
107/26633
108/26633
109/26633
110/26633
111/2663

832/26633
833/26633
834/26633
835/26633
836/26633
837/26633
838/26633
839/26633
840/26633
841/26633
842/26633
843/26633
844/26633
845/26633
846/26633
847/26633
848/26633
849/26633
850/26633
851/26633
852/26633
853/26633
854/26633
855/26633
856/26633
857/26633
858/26633
859/26633
860/26633
861/26633
862/26633
863/26633
864/26633
865/26633
866/26633
867/26633
868/26633
869/26633
870/26633
871/26633
872/26633
873/26633
874/26633
875/26633
876/26633
877/26633
878/26633
879/26633
880/26633
881/26633
882/26633
883/26633
884/26633
885/26633
886/26633
887/26633
888/26633
889/26633
890/26633
891/26633
892/26633
893/26633
894/26633
895/26633
896/26633
897/26633
898/26633
899/26633
900/26633
901/26633
902/26633
903/26633
904/26633
905/26633
906/26633
907/26633
908/26633
909/26633
910/26633
911/26633
912/26633
913/26633
914/26633
915/26633
916/26633
917/26633
918/26633
919/26633
920/26633
921/26633
922/26633
923/26633
924/26633
925/26633
926/26633
927/26633
928/26633
929/26633
930/26633
931/26633


1593/26633
1594/26633
1595/26633
1596/26633
1597/26633
1598/26633
1599/26633
1600/26633
1601/26633
1602/26633
1603/26633
1604/26633
1605/26633
1606/26633
1607/26633
1608/26633
1609/26633
1610/26633
1611/26633
1612/26633
1613/26633
1614/26633
1615/26633
1616/26633
1617/26633
1618/26633
1619/26633
1620/26633
1621/26633
1622/26633
1623/26633
1624/26633
1625/26633
1626/26633
1627/26633
1628/26633
1629/26633
1630/26633
1631/26633
1632/26633
1633/26633
1634/26633
1635/26633
1636/26633
1637/26633
1638/26633
1639/26633
1640/26633
1641/26633
1642/26633
1643/26633
1644/26633
1645/26633
1646/26633
1647/26633
1648/26633
1649/26633
1650/26633
1651/26633
1652/26633
1653/26633
1654/26633
1655/26633
1656/26633
1657/26633
1658/26633
1659/26633
1660/26633
1661/26633
1662/26633
1663/26633
1664/26633
1665/26633
1666/26633
1667/26633
1668/26633
1669/26633
1670/26633
1671/26633
1672/26633
1673/26633
1674/26633
1675/26633
1676/26633
1677/26633
1678/26633
1679/26633
1680/26633
1681/26633
1682/26633
1683/26633

2341/26633
2342/26633
2343/26633
2344/26633
2345/26633
2346/26633
2347/26633
2348/26633
2349/26633
2350/26633
2351/26633
2352/26633
2353/26633
2354/26633
2355/26633
2356/26633
2357/26633
2358/26633
2359/26633
2360/26633
2361/26633
2362/26633
2363/26633
2364/26633
2365/26633
2366/26633
2367/26633
2368/26633
2369/26633
2370/26633
2371/26633
2372/26633
2373/26633
2374/26633
2375/26633
2376/26633
2377/26633
2378/26633
2379/26633
2380/26633
2381/26633
2382/26633
2383/26633
2384/26633
2385/26633
2386/26633
2387/26633
2388/26633
2389/26633
2390/26633
2391/26633
2392/26633
2393/26633
2394/26633
2395/26633
2396/26633
2397/26633
2398/26633
2399/26633
2400/26633
2401/26633
2402/26633
2403/26633
2404/26633
2405/26633
2406/26633
2407/26633
2408/26633
2409/26633
2410/26633
2411/26633
2412/26633
2413/26633
2414/26633
2415/26633
2416/26633
2417/26633
2418/26633
2419/26633
2420/26633
2421/26633
2422/26633
2423/26633
2424/26633
2425/26633
2426/26633
2427/26633
2428/26633
2429/26633
2430/26633
2431/26633

...

24210/26633
24211/26633
24212/26633
24213/26633
24214/26633
24215/26633
24216/26633
24217/26633
24218/26633
24219/26633
24220/26633
24221/26633
24222/26633
24223/26633
24224/26633
24225/26633
24226/26633
24227/26633
24228/26633
24229/26633
24230/26633
24231/26633
24232/26633
24233/26633
24234/26633
24235/26633
24236/26633
24237/26633
24238/26633
24239/26633
24240/26633
24241/26633
24242/26633
24243/26633
24244/26633
24245/26633
24246/26633
24247/26633
24248/26633
24249/26633
24250/26633
24251/26633
24252/26633
24253/26633
24254/26633
24255/26633
24256/26633
24257/26633
24258/26633
24259/26633
24260/26633
24261/26633
24262/26633
24263/26633
24264/26633
24265/26633
24266/26633
24267/26633
24268/26633
24269/26633
24270/26633
24271/26633
24272/26633
24273/26633
24274/26633
24275/26633
24276/26633
24277/26633
24278/26633
24279/26633
24280/26633
24281/26633
24282/26633
24283/26633
24284/26633
24285/26633
24286/26633
24287/26633
24288/26633
24289/26633
24290/26633
24291/26633
24292/26633
2429

24893/26633
24894/26633
24895/26633
24896/26633
24897/26633
24898/26633
24899/26633
24900/26633
24901/26633
24902/26633
24903/26633
24904/26633
24905/26633
24906/26633
24907/26633
24908/26633
24909/26633
24910/26633
24911/26633
24912/26633
24913/26633
24914/26633
24915/26633
24916/26633
24917/26633
24918/26633
24919/26633
24920/26633
24921/26633
24922/26633
24923/26633
24924/26633
24925/26633
24926/26633
24927/26633
24928/26633
24929/26633
24930/26633
24931/26633
24932/26633
24933/26633
24934/26633
24935/26633
24936/26633
24937/26633
24938/26633
24939/26633
24940/26633
24941/26633
24942/26633
24943/26633
24944/26633
24945/26633
24946/26633
24947/26633
24948/26633
24949/26633
24950/26633
24951/26633
24952/26633
24953/26633
24954/26633
24955/26633
24956/26633
24957/26633
24958/26633
24959/26633
24960/26633
24961/26633
24962/26633
24963/26633
24964/26633
24965/26633
24966/26633
24967/26633
24968/26633
24969/26633
24970/26633
24971/26633
24972/26633
24973/26633
24974/26633
24975/26633
2497

25579/26633
25580/26633
25581/26633
25582/26633
25583/26633
25584/26633
25585/26633
25586/26633
25587/26633
25588/26633
25589/26633
25590/26633
25591/26633
25592/26633
25593/26633
25594/26633
25595/26633
25596/26633
25597/26633
25598/26633
25599/26633
25600/26633
25601/26633
25602/26633
25603/26633
25604/26633
25605/26633
25606/26633
25607/26633
25608/26633
25609/26633
25610/26633
25611/26633
25612/26633
25613/26633
25614/26633
25615/26633
25616/26633
25617/26633
25618/26633
25619/26633
25620/26633
25621/26633
25622/26633
25623/26633
25624/26633
25625/26633
25626/26633
25627/26633
25628/26633
25629/26633
25630/26633
25631/26633
25632/26633
25633/26633
25634/26633
25635/26633
25636/26633
25637/26633
25638/26633
25639/26633
25640/26633
25641/26633
25642/26633
25643/26633
25644/26633
25645/26633
25646/26633
25647/26633
25648/26633
25649/26633
25650/26633
25651/26633
25652/26633
25653/26633
25654/26633
25655/26633
25656/26633
25657/26633
25658/26633
25659/26633
25660/26633
25661/26633
2566

26266/26633
26267/26633
26268/26633
26269/26633
26270/26633
26271/26633
26272/26633
26273/26633
26274/26633
26275/26633
26276/26633
26277/26633
26278/26633
26279/26633
26280/26633
26281/26633
26282/26633
26283/26633
26284/26633
26285/26633
26286/26633
26287/26633
26288/26633
26289/26633
26290/26633
26291/26633
26292/26633
26293/26633
26294/26633
26295/26633
26296/26633
26297/26633
26298/26633
26299/26633
26300/26633
26301/26633
26302/26633
26303/26633
26304/26633
26305/26633
26306/26633
26307/26633
26308/26633
26309/26633
26310/26633
26311/26633
26312/26633
26313/26633
26314/26633
26315/26633
26316/26633
26317/26633
26318/26633
26319/26633
26320/26633
26321/26633
26322/26633
26323/26633
26324/26633
26325/26633
26326/26633
26327/26633
26328/26633
26329/26633
26330/26633
26331/26633
26332/26633
26333/26633
26334/26633
26335/26633
26336/26633
26337/26633
26338/26633
26339/26633
26340/26633
26341/26633
26342/26633
26343/26633
26344/26633
26345/26633
26346/26633
26347/26633
26348/26633
2634

/tmp/ipykernel_12528/2246424491.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zips_df['longitude']=longitude
/tmp/ipykernel_12528/2246424491.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zips_df['latitude']=latitude
